In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
import gc
import random

import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings(action='ignore')

from pycaret.classification import *

import random as python_random
np.random.seed(42)
python_random.seed(42)
set_config('seed', 42)

In [2]:
PATH = './data/'

train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_qua = pd.read_csv(PATH+'train_quality_data.csv')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_qua  = pd.read_csv(PATH+'test_quality_data.csv')

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
train_err

In [3]:
train_err['date'] = train_err['time'].astype(str).str.slice(0,8)
train_err['model_fwver'] = train_err.model_nm + train_err.fwver
train_err['errtype_code'] = train_err.errtype.astype(str) + train_err.errcode

train_err_0 = train_err[['user_id','date','model_fwver','errtype_code']].drop_duplicates()
display(train_err_0.head())

test_err['date'] = test_err['time'].astype(str).str.slice(0,8)
test_err['model_fwver'] = test_err.model_nm + test_err.fwver
test_err['errtype_code'] = test_err.errtype.astype(str) + test_err.errcode

test_err_0 = test_err[['user_id','date','model_fwver','errtype_code']].drop_duplicates()
display(test_err_0.head())

,user_id,date,model_fwver,errtype_code
0,10000,20201101,model_305.15.2138,151
1,10000,20201101,model_305.15.2138,121
2,10000,20201101,model_305.15.2138,111
3,10000,20201101,model_305.15.2138,161
4,10000,20201101,model_305.15.2138,40


,user_id,date,model_fwver,errtype_code
0,30000,20201101,model_104.16.3553,311
1,30000,20201101,model_104.16.3553,332
2,30000,20201101,model_104.16.3553,151
3,30000,20201101,model_104.16.3553,221
4,30000,20201101,model_104.16.3553,111


In [ ]:
train_err.shape, train_err_0.shape

In [4]:
train_err_1 = train_err.groupby(['user_id','date']).count().reset_index()[['user_id','date','time']]
train_err_1.columns = ['user_id','date','date_cnt']
train_err_1 = train_err_1.groupby('user_id').agg({'date':'count','date_cnt': 'sum'}).reset_index()
train_err_1.columns = ['user_id','date_cnt', 'date_sum']
display(train_err_1.head())

test_err_1 = test_err.groupby(['user_id','date']).count().reset_index()[['user_id','date','time']]
test_err_1.columns = ['user_id','date','date_cnt']
test_err_1 = test_err_1.groupby('user_id').agg({'date':'count','date_cnt': 'sum'}).reset_index()
test_err_1.columns = ['user_id','date_cnt', 'date_sum']
display(test_err_1.head())

train_err_1.shape

,user_id,date_cnt,date_sum
0,10000,30,317
1,10001,30,2365
2,10002,29,306
3,10003,30,306
4,10004,30,777


,user_id,date_cnt,date_sum
0,30000,29,2750
1,30001,28,284
2,30002,30,941
3,30003,28,371
4,30004,30,881


(15000, 3)

In [5]:
train = train_err_1.copy()
test = test_err_1.copy()

In [6]:
train_model = set(train_err.model_fwver.unique())
test_model = set(test_err.model_fwver.unique())
models = train_model & test_model
models = list(models)

In [7]:
for model in models:
    train[model] = 0
    test[model] = 0
    
train.shape, test.shape    

((15000, 34), (14998, 34))

In [ ]:
train

In [8]:
train_err_2 = train_err.groupby(['user_id','model_fwver']).count().reset_index()[['user_id','model_fwver','time']]
train_err_2.columns = ['user_id','model_fwver','model_fwver_cnt']
#train_err_2['model_fwver_rank'] = train_err_2.model_fwver_cnt.rank()
display(train_err_2.head())

test_err_2 = test_err.groupby(['user_id','model_fwver']).count().reset_index()[['user_id','model_fwver','time']]
test_err_2.columns = ['user_id','model_fwver','model_fwver_cnt']
#test_err_2['model_fwver_rank'] = test_err_2.model_fwver_cnt.rank()
display(test_err_2.head())

,user_id,model_fwver,model_fwver_cnt
0,10000,model_305.15.2138,317
1,10001,model_204.33.1185,379
2,10001,model_204.33.1261,1986
3,10002,model_305.15.2138,306
4,10003,model_204.33.1185,81


,user_id,model_fwver,model_fwver_cnt
0,30000,model_104.16.3553,2320
1,30000,model_104.16.3571,367
2,30000,model_204.33.1261,63
3,30001,model_305.15.2138,284
4,30002,model_004.22.1750,733


In [9]:
for idx, col in tqdm(enumerate(train_err_2.values)):
    i = train[train.user_id == train_err_2.loc[idx].user_id].index
    c = train.columns[train.columns == train_err_2.loc[idx].model_fwver]
    
    train.loc[i, c] += train_err_2.loc[idx].model_fwver_cnt    

for idx, col in tqdm(enumerate(test_err_2.values)):
    i = test[test.user_id == test_err_2.loc[idx].user_id].index
    c = test.columns[test.columns == test_err_2.loc[idx].model_fwver]
    
    test.loc[i, c] += test_err_2.loc[idx].model_fwver_cnt    

24062it [01:08, 353.13it/s]
24096it [01:07, 354.79it/s]


In [10]:
train_error = set(train_err.errtype.unique())
test_error = set(test_err.errtype.unique())
errors = train_error & test_error
errors = list(errors)

In [11]:
for error in errors:
    train[error] = 0
    test[error] = 0
    
train.shape, test.shape    

((15000, 75), (14998, 75))

In [12]:
# train_err > errtype => rank
train_err_3 = train_err.groupby(['user_id','errtype']).count().reset_index()[['user_id','errtype','time']]
train_err_3.columns = ['user_id','errtype','errtype_cnt']
#train_err_3['errtype_rank'] = train_err_3.errtype_cnt.rank()
display(train_err_3.head())

# test_err > errtype => rank
test_err_3 = test_err.groupby(['user_id','errtype']).count().reset_index()[['user_id','errtype','time']]
test_err_3.columns = ['user_id','errtype','errtype_cnt']
#test_err_3['errtype_rank'] = test_err_3.errtype_cnt.rank()
display(test_err_3.head())

,user_id,errtype,errtype_cnt
0,10000,3,8
1,10000,4,104
2,10000,6,1
3,10000,7,1
4,10000,10,7


,user_id,errtype,errtype_cnt
0,30000,5,62
1,30000,6,1
2,30000,7,1
3,30000,11,16
4,30000,12,16


In [13]:
for idx, col in tqdm(enumerate(train_err_3.values)):
    i = train[train.user_id == train_err_3.loc[idx].user_id].index
    c = train.columns[train.columns == train_err_3.loc[idx].errtype]
    
    train.loc[i, c] += train_err_3.loc[idx].errtype_cnt    

for idx, col in tqdm(enumerate(test_err_3.values)):
    i = test[test.user_id == test_err_3.loc[idx].user_id].index
    c = test.columns[test.columns == test_err_3.loc[idx].errtype]
    
    test.loc[i, c] += test_err_3.loc[idx].errtype_cnt    

231039it [11:08, 345.52it/s]
231241it [11:05, 347.58it/s]


In [14]:
train

,user_id,date_cnt,date_sum,model_104.16.3569,model_204.33.1261,model_104.16.3553,model_403.11.1141,model_204.33.1149,model_403.11.1167,model_204.33.1171,...,33,34,35,36,37,38,39,40,41,42
0,10000,30,317,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,30,2365,0,1986,0,0,0,0,0,...,10,18,0,1,1,0,0,113,56,1
2,10002,29,306,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,30,306,0,225,0,0,0,0,0,...,8,0,0,1,1,2,0,17,1,0
4,10004,30,777,0,0,0,0,0,0,0,...,16,0,0,1,1,0,0,4,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,10,194,0,194,0,0,0,0,0,...,5,0,0,0,0,0,0,9,7,4
14996,24996,1,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14997,24997,30,826,0,0,0,0,0,0,0,...,16,17,0,1,1,0,0,58,8,5
14998,24998,21,155,0,0,0,0,0,0,0,...,15,12,0,1,1,0,0,6,0,0


In [ ]:
# train_err > errtype_code => rank
train_err_4 = train_err.groupby(['model_fwver','errtype_code']).count().reset_index()[['model_fwver','errtype_code','user_id']]
train_err_4.columns = ['model_fwver','errtype_code','cnt']
train_err_4['rank'] = train_err_4.cnt.rank()
display(train_err_4.head())

In [ ]:
# test_err > errtype_code => rank
test_err_4 = test_err.groupby(['model_fwver','errtype_code']).count().reset_index()[['model_fwver','errtype_code','user_id']]
test_err_4.columns = ['model_fwver','errtype_code','cnt']
test_err_4['rank'] = test_err_4.cnt.rank()
display(test_err_4.head())

In [ ]:
train = train_err.merge(train_err_1, on=['user_id'], how='left').merge(train_err_4, on=['model_fwver','errtype_code'], how='left')#.merge(train_err_3, on='errtype_code', how='left')
train = train[['user_id','date_cnt','date_sum','model_fwver','errtype_code','cnt','rank']].drop_duplicates()
train = train.groupby(['user_id','date_cnt','date_sum']).agg({'errtype_code':['count'],'cnt':['sum'],'rank':['max']}).reset_index()
train.columns = ['user_id','date_cnt','date_sum','errtype_code_cnt','cnt_sum','rank_max']
train

In [ ]:
test = test_err.merge(test_err_1, on=['user_id'], how='left').merge(test_err_4, on=['model_fwver','errtype_code'], how='left')#.merge(test_err_3, on='errtype_code', how='left')
test = test[['user_id','date_cnt','date_sum','model_fwver','errtype_code','cnt','rank']].drop_duplicates()
test = test.groupby(['user_id','date_cnt','date_sum']).agg({'errtype_code':['count'],'cnt':['sum'],'rank':['max']}).reset_index()
test.columns = ['user_id','date_cnt','date_sum','errtype_code_cnt','cnt_sum','rank_max']
test

In [20]:
train_qua_0 = train_qua.drop_duplicates()
test_qua_0 = test_qua.drop_duplicates()

train_qua_0.drop(['quality_3','quality_4','time','fwver'], axis=1, inplace=True)
test_qua_0.drop(['quality_3','quality_4','time','fwver'], axis=1, inplace=True)

train_qua_0.fillna(0, inplace=True)
test_qua_0.fillna(0, inplace=True)

In [16]:
def chg_qua(x):
    if x == 0:
        return 0
    else:
        return 1

In [21]:
cols = ['quality_0', 'quality_1', 'quality_2', 'quality_5','quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10','quality_11', 'quality_12']

for col in cols:
    train_qua_0[col] = train_qua_0[col].apply(chg_qua)
    test_qua_0[col] = test_qua_0[col].apply(chg_qua)

In [22]:
train_qua_1 = train_qua_0.groupby('user_id').sum().reset_index()
test_qua_1 = test_qua_0.groupby('user_id').sum().reset_index()

In [23]:
train_qua_1

,user_id,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,0,0,0,4,0,4,0,4,4,0,0
1,10002,2,1,2,22,4,22,0,22,22,1,0
2,10004,1,1,1,6,2,6,0,6,6,1,0
3,10005,1,1,1,6,3,6,0,6,6,1,0
4,10006,0,0,0,6,1,6,0,6,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8276,24990,0,1,1,5,1,0,0,0,5,1,0
8277,24992,2,2,2,9,2,0,0,0,9,2,0
8278,24993,2,2,2,6,2,0,0,0,6,2,0
8279,24995,2,2,2,6,2,0,0,0,6,2,0


In [19]:
%%time
for col in cols:
    train_qua_1[col] = train_qua_1[col].apply(chg_qua)
    test_qua_1[col] = test_qua_1[col].apply(chg_qua)

Wall time: 61.4 ms


In [ ]:
cols=['quality_0', 'quality_1','quality_2','quality_5',  'quality_6','quality_7', 'quality_10',]

In [ ]:
train_qua_1.drop(cols, axis=1, inplace=True)
test_qua_1.drop(cols, axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(train_qua_1.corr(), annot=True, fmt='.2f', cmap='RdYlGn_r')

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(test_qua_1.corr(), annot=True, fmt='.2f', cmap='RdYlGn_r')

In [24]:
train = train.merge(train_qua_1, on='user_id', how='left').fillna(0)
test = test.merge(test_qua_1, on='user_id', how='left').fillna(0)

In [25]:
train_prob1 = train_prob.groupby('user_id').count().reset_index()
train_prob1.time = train_prob1.time.apply(chg_qua)
train_prob1.columns = ['user_id', 'prob']
train_prob1

,user_id,prob
0,10001,1
1,10004,1
2,10005,1
3,10006,1
4,10008,1
...,...,...
4995,24983,1
4996,24984,1
4997,24990,1
4998,24997,1


In [26]:
train = train.merge(train_prob1, on='user_id', how='left').fillna(0)

In [27]:
display(train.head())
display(test.head())

,user_id,date_cnt,date_sum,model_104.16.3569,model_204.33.1261,model_104.16.3553,model_403.11.1141,model_204.33.1149,model_403.11.1167,model_204.33.1171,...,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,prob
0,10000,30,317,0,0,0,0,0,0,0,...,0.0,4.0,0.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0
1,10001,30,2365,0,1986,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,10002,29,306,0,0,0,0,0,0,0,...,2.0,22.0,4.0,22.0,0.0,22.0,22.0,1.0,0.0,0.0
3,10003,30,306,0,225,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,30,777,0,0,0,0,0,0,0,...,1.0,6.0,2.0,6.0,0.0,6.0,6.0,1.0,0.0,1.0


,user_id,date_cnt,date_sum,model_104.16.3569,model_204.33.1261,model_104.16.3553,model_403.11.1141,model_204.33.1149,model_403.11.1167,model_204.33.1171,...,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,29,2750,0,63,2320,0,0,0,0,...,0.0,0.0,2.0,1.0,2.0,0.0,2.0,2.0,0.0,0.0
1,30001,28,284,0,0,0,0,0,0,0,...,1.0,1.0,10.0,1.0,10.0,0.0,10.0,10.0,1.0,0.0
2,30002,30,941,0,0,0,0,0,0,0,...,3.0,3.0,26.0,3.0,26.0,0.0,26.0,26.0,3.0,0.0
3,30003,28,371,0,0,0,0,0,0,0,...,0.0,0.0,13.0,5.0,13.0,0.0,13.0,13.0,0.0,0.0
4,30004,30,881,0,255,623,0,3,0,0,...,1.0,1.0,5.0,3.0,5.0,0.0,5.0,5.0,1.0,0.0


In [ ]:
train.shape, test.shape

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(train.corr(), annot=True, fmt='.2f', cmap='RdYlGn_r')

In [ ]:
test.columns

In [ ]:
cols = ['date_cnt', 'date_sum', 'model_fwver_cnt', 'model_fwver_max','errtype_code_max']

In [ ]:
min = train[cols].min()
max  = train[cols].max()

In [ ]:
for i, col in enumerate(cols):
    train[col] = (train[col] - min[i]) / (max[i] - min[i])
    test[col] = (test[col] - min[i]) / (max[i] - min[i])

In [28]:
train.prob = train.prob.astype(int)

In [39]:
cols = ['quality_0', 'quality_1', 'quality_2', 'quality_5','quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10','quality_11', 'quality_12']
train[cols] = train[cols].astype(int)
test[cols] = test[cols].astype(int)

In [40]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 14999
Data columns (total 87 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   user_id            15000 non-null  int64
 1   date_cnt           15000 non-null  int64
 2   date_sum           15000 non-null  int64
 3   model_104.16.3569  15000 non-null  int64
 4   model_204.33.1261  15000 non-null  int64
 5   model_104.16.3553  15000 non-null  int64
 6   model_403.11.1141  15000 non-null  int64
 7   model_204.33.1149  15000 non-null  int64
 8   model_403.11.1167  15000 non-null  int64
 9   model_204.33.1171  15000 non-null  int64
 10  model_004.22.1666  15000 non-null  int64
 11  model_004.22.1684  15000 non-null  int64
 12  model_305.15.2138  15000 non-null  int64
 13  model_705.66.3237  15000 non-null  int64
 14  model_610          15000 non-null  int64
 15  model_305.15.2092  15000 non-null  int64
 16  model_305.15.2114  15000 non-null  int64
 17  model_804.73

In [41]:
train.columns[1:86]

Index(['date_cnt', 'date_sum', 'model_104.16.3569', 'model_204.33.1261',
       'model_104.16.3553', 'model_403.11.1141', 'model_204.33.1149',
       'model_403.11.1167', 'model_204.33.1171', 'model_004.22.1666',
       'model_004.22.1684', 'model_305.15.2138', 'model_705.66.3237',
       'model_610', 'model_305.15.2092', 'model_305.15.2114',
       'model_804.73.2237', 'model_104.16.3571', 'model_305.15.3104',
       'model_004.22.1656', 'model_504.82.1730', 'model_104.16.3439',
       'model_403.11.1149', 'model_504.82.1684', 'model_004.22.1778',
       'model_204.33.1185', 'model_204.33.1125', 'model_305.15.2120',
       'model_504.82.1778', 'model_705.66.3571', 'model_804.73.2571',
       'model_004.22.1750', 'model_68.5.3', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27', '28', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 'qu

In [42]:
%%time
clf = setup(session_id=42, data=train, target='prob'
           , numeric_features=train.columns[1:86])

Setup Succesfully Completed!


,Description,Value
0,session_id,42
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(15000, 87)"
4,Missing Values,False
5,Numeric Features,86
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


Wall time: 4.85 s


In [43]:
best = compare_models(sort = 'AUC', n_select = 5
                     ,exclude=['lr', 'knn', 'nb', 'dt', 'svm', 'rbfsvm', 'ridge','qda']
                     )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,CatBoost Classifier,0.7951,0.8164,0.5051,0.8087,0.6217,0.4913,0.5170,12.0292
1,Light Gradient Boosting Machine,0.7926,0.8120,0.5163,0.7885,0.6238,0.4892,0.5102,0.4813
2,Gradient Boosting Classifier,0.7865,0.8112,0.4669,0.8134,0.5930,0.4623,0.4952,3.3968
3,Extra Trees Classifier,0.7842,0.8007,0.4909,0.7803,0.6026,0.4648,0.4883,0.6440
4,Ada Boost Classifier,0.7772,0.7960,0.4909,0.7552,0.5948,0.4506,0.4706,0.8986
5,Extreme Gradient Boosting,0.7812,0.7959,0.5349,0.7368,0.6195,0.4714,0.4835,1.8167
6,Random Forest Classifier,0.7649,0.7725,0.4569,0.7380,0.5641,0.4153,0.4377,0.1227
7,Linear Discriminant Analysis,0.7505,0.7378,0.3297,0.8079,0.4682,0.3410,0.3995,0.1510


In [44]:
blended = blend_models(estimator_list = best, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7833,0.8050,0.4671,0.7995,0.5897,0.4560,0.4863
1,0.8043,0.8306,0.5200,0.8292,0.6392,0.5144,0.5407
2,0.7895,0.8152,0.4843,0.8071,0.6054,0.4738,0.5026
3,0.7981,0.8286,0.4800,0.8485,0.6131,0.4904,0.5268
4,0.7847,0.7998,0.4814,0.7911,0.5986,0.4631,0.4898
Mean,0.7920,0.8158,0.4866,0.8151,0.6092,0.4795,0.5092
SD,0.0080,0.0123,0.0177,0.0210,0.0169,0.0209,0.0212


In [45]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7918,0.8172,0.4847,0.8159,0.6081,0.4786,0.5087


In [46]:
%%time
final_model = finalize_model(blended)

Wall time: 3min 16s


In [47]:
test_x = sample_submssion[['user_id']].merge(test, on='user_id', how='left').fillna(0)
test_x.shape

(14999, 86)

In [48]:
predictions = predict_model(final_model, data = test_x)

In [49]:
sample_submssion['problem'] = predictions['Score']
sample_submssion.head()

,user_id,problem
0,30000,0.8220
1,30001,0.2632
2,30002,0.3450
3,30003,0.7602
4,30004,0.7947


In [50]:
sample_submssion.to_csv("./submission/submission_20210128-2.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.8220
1,30001,0.2632
2,30002,0.3450
3,30003,0.7602
4,30004,0.7947
...,...,...
14994,44994,0.3070
14995,44995,0.2883
14996,44996,0.5472
14997,44997,0.7624
